In [18]:
import ee
import os
import osmnx as ox
import geopandas as gpd
import pandas as pd
import geemap


import corineFunctions

In [19]:
download_dir = os.path.join(os.path.expanduser("~"), "GitHub\GEE_Landcover\Figures")
if not os.path.exists(download_dir):
    os.makedirs(download_dir)

In [27]:
ee.Authenticate()

True

In [28]:
ee.Initialize() # optionally add project id: project='my-project-id'

In [21]:
# Enter city name
city_name = "Bayreuth, Germany"

# Retrieve the boundary
city_boundary = ox.geocode_to_gdf(city_name) #to geopandas dataframe
roi = geemap.geopandas_to_ee(city_boundary).geometry() #to ee geometry


In [22]:
corineMap = corineFunctions.visualize_corine_land_cover(roi, map_title="CORINE Classes in Study Area")
corineMap

Map(center=[49.93779121887736, 11.584775998082916], controls=(WidgetControl(options=['position', 'transparent_…

In [23]:
# save map to file
html_file = os.path.join(download_dir, "corine-llc.html")
corineMap.to_html(filename=html_file, title="Query CORINE Landcover using GEE", width="100%", height="880px")


In [24]:
# # Load the CORINE dataset and clip to the ROI
corine = ee.Image("COPERNICUS/CORINE/V20/100m/2018")
corine_clipped = corine.clip(roi)
class_names = corine.get("landcover_class_names").getInfo()
class_values = corine.get("landcover_class_values").getInfo()
class_palette = corine.get("landcover_class_palette").getInfo()

# Create mappings from class value to human-readable name and its corresponding color.
class_mapping = {int(val): class_names[i].split("; ")[-1] for i, val in enumerate(class_values)}
color_mapping = {int(val): class_palette[i] for i, val in enumerate(class_values)}

In [25]:
df = corineFunctions.compute_landcover_area(corine_clipped, roi, class_mapping, verbose=False)


In [26]:
df

Class                      Class Name    Area (ha)
0    111         continuous urban fabric   170.079031
1    112      discontinuous urban fabric  1563.459485
2    121  industrial or commercial units   658.919045
3    142    sport and leisure facilities   141.991708
4    211       non-irrigated arable land  1357.786450
5    231                        pastures  1467.411379
6    242    complex cultivation patterns    92.181648
7    311             broad-leaved forest    80.657639
8    312               coniferous forest   999.426860
9    313                    mixed forest   157.506952

<!-- ### Add Sentinel-2 Layers -->